你可以分别使用 Esc 和 Enter 在命令模式和编辑模式之间跳跃。现在就试试看吧！
进入命令模式之后（此时你没有活跃单元），你可以尝试以下快捷键：
A 会在活跃单元之上插入一个新的单元，B 会在活跃单元之下插入一个新单元。
连续按两次 D，可以删除一个单元。
撤销被删除的单元，按 Z。
Y 会将当前活跃的单元变成一个代码单元。
按住 Shift +上或下箭头可选择多个单元。在多选模式时，按住 Shift + M 可合并你的选择。
按 F 会弹出「查找和替换」菜单。
处于编辑模式时（在命令模式时按 Enter 会进入编辑模式），你会发现下列快捷键很有用：
Ctrl + Home 到达单元起始位置。
Ctrl + S 保存进度。
如之前提到的，Ctrl + Enter 会运行你的整个单元块。
Alt + Enter 不止会运行你的单元块，还会在下面添加一个新单元。
Ctrl + Shift + F 打开命令面板。
要查看键盘快捷键完整列表，可在命令模式按「H」或进入「Help > Keyboard Shortcuts」。你一定要经常看这些快捷键，因为常会添加新的。

In [1]:
%matplotlib inline
import d2lzh as d2l
from mxnet import gluon, image, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import numpy as np
import sys

In [12]:
def bilinear_kernel(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size),
                      dtype='float32')
    weight[range(in_channels), range(out_channels), :, :] = filt
    return nd.array(weight)

下面我们使用一个基于ImageNet数据集预训练的ResNet-18模型来抽取图像特征，并将该网络实例记为pretrained_net。可以看到，该模型成员变量features的最后两层分别是全局最大池化层GlobalAvgPool2D和样本变平层Flatten，而output模块包含了输出用的全连接层。全卷积网络不需要使用这些层。

In [6]:
pretrained_net = model_zoo.vision.resnet18_v2(pretrained=True)
pretrained_net.features[-4:], pretrained_net.output

(HybridSequential(
   (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=512)
   (1): Activation(relu)
   (2): GlobalAvgPool2D(size=(1, 1), stride=(1, 1), padding=(0, 0), ceil_mode=True)
   (3): Flatten
 ), Dense(512 -> 1000, linear))

In [10]:
net = nn.HybridSequential()#512通道输出，缩小1/32
for layer in pretrained_net.features[:-2]:#去除池化和展平层
    net.add(layer)

In [18]:
num_classes = 21
net.add(nn.Conv2D(num_classes, kernel_size=1),#增加1*1分类和转置卷积输出，
        nn.Conv2DTranspose(num_classes, kernel_size=64, padding=16,strides=32))#还原大小，但是为21通道

In [19]:
net[-1].initialize(init.Constant(bilinear_kernel(num_classes, num_classes,64)))#采用双线性插值还原
net[-2].initialize(init=init.Xavier())

In [ ]:
crop_size, batch_size, colormap2label = (320, 480), 32, nd.zeros(256**3)
for i, cm in enumerate(d2l.VOC_COLORMAP):
    colormap2label[(cm[0] * 256 + cm[1]) * 256 + cm[2]] = i
voc_dir = d2l.download_voc_pascal(data_dir='../data')

num_workers = 0 if sys.platform.startswith('win32') else 4
train_iter = gdata.DataLoader(
    d2l.VOCSegDataset(True, crop_size, voc_dir, colormap2label), batch_size,
    shuffle=True, last_batch='discard', num_workers=num_workers)
test_iter = gdata.DataLoader(
    d2l.VOCSegDataset(False, crop_size, voc_dir, colormap2label), batch_size,
    last_batch='discard', num_workers=num_workers)

In [ ]:
ctx = d2l.try_all_gpus()
loss = gloss.SoftmaxCrossEntropyLoss(axis=1)
net.collect_params().reset_ctx(ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1,
                                                      'wd': 1e-3})
d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs=5)

In [ ]:
def predict(img):
    X = test_iter._dataset.normalize_image(img)
    X = X.transpose((2, 0, 1)).expand_dims(axis=0)
    pred = nd.argmax(net(X.as_in_context(ctx[0])), axis=1)#将21通道转成1通道，代表该点的类别
    return pred.reshape((pred.shape[1], pred.shape[2]))

In [ ]:
def label2image(pred):#将预测类别映射回它们在数据集中的标注颜色，将1通道类数据变成3通道颜色数据
    colormap = nd.array(d2l.VOC_COLORMAP, ctx=ctx[0], dtype='uint8')
    X = pred.astype('int32')
    return colormap[X, :]

In [ ]:
test_images, test_labels = d2l.read_voc_images(is_train=False)#读取测试集样本
n, imgs = 4, []
for i in range(n):
    crop_rect = (0, 0, 480, 320)
    X = image.fixed_crop(test_images[i], *crop_rect)#裁剪测试集
    pred = label2image(predict(X))#将预测类别映射回它们在数据集中的标注颜色
    imgs += [X, pred, image.fixed_crop(test_labels[i], *crop_rect)]#组成图片
d2l.show_images(imgs[::3] + imgs[1::3] + imgs[2::3], 3, n);